In [1]:
#import the libraries 
import numpy as np 
import tensorflow as tf 

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import TimeDistributed, Dense, Dropout, SimpleRNN, RepeatVector
from tensorflow.keras.callbacks import EarlyStopping, LambdaCallback

from termcolor import colored

In [2]:
#Generate Data 
all_chars = '0123456789+' 

In [4]:
num_features = len(all_chars)
print('Number of features : ', num_features)
char_to_index = dict((c, i) for i, c in enumerate(all_chars))
index_to_char = dict((i, c) for i, c in enumerate(all_chars))

Number of features :  11


In [5]:
def generate_data():
    first = np.random.randint(0, 100)
    second = np.random.randint(0,100)
    example = str(first) + '+' + str(second)
    label = str(first + second)
    return example, label
generate_data()

('36+96', '132')

In [9]:
#Create the Model
hidden_units = 128
max_time_steps = 5

model = Sequential([
    #encoder
    SimpleRNN(hidden_units, input_shape = (None, num_features)), 
    RepeatVector(max_time_steps), 
    #decoder
    SimpleRNN(hidden_units, return_sequences = True),
    TimeDistributed(Dense(num_features, activation = 'softmax'))
])

model.compile(
    loss = 'categorical_crossentropy', 
    optimizer = 'adam', 
    metrics = ['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_4 (SimpleRNN)     (None, 128)               17920     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 128)            0         
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, 5, 128)            32896     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 11)             1419      
Total params: 52,235
Trainable params: 52,235
Non-trainable params: 0
_________________________________________________________________


In [13]:
#Vectorize and De-Vectorize the Data 
def vectorize_example(example, label):
    x = np.zeros((max_time_steps, num_features))
    y = np.zeros((max_time_steps, num_features))
    
    diff_x = max_time_steps - len(example)
    diff_y = max_time_steps - len(label)    
    
    for i, c in enumerate(example):
        x[i+diff_x, char_to_index[c]] = 1
    for i in range(diff_x):
        x[i, char_to_index['0']] = 1
        
    for i, c in enumerate(label):
        y[i+diff_y, char_to_index[c]] = 1
    for i in range(diff_y):
        y[i, char_to_index['0']] = 1
    return x, y

e, l = generate_data()
print(e, l)
x, y = vectorize_example(e, l)
print(x.shape, y.shape)

18+67 85
(5, 11) (5, 11)


In [14]:
 def devectorize_example(example):
        result = [index_to_char[np.argmax(vec)] for i, vec in enumerate(example)]
        return ''.join(result)
    
devectorize_example(x)
        

'18+67'

In [15]:
devectorize_example(y)

'00085'

In [16]:
#Create Dataset
def create_dataset(num_examples=2000):

    x_train = np.zeros((num_examples, max_time_steps, num_features))
    y_train = np.zeros((num_examples, max_time_steps, num_features))

    for i in range(num_examples):
        e, l = generate_data()
        x, y = vectorize_example(e, l)
        x_train[i] = x
        y_train[i] = y
    
    return x_train, y_train

x_train, y_train = create_dataset(20000)
print(x_train.shape, y_train.shape)

(20000, 5, 11) (20000, 5, 11)


In [17]:
devectorize_example(x_train[0])

'83+98'

In [18]:
devectorize_example(y_train[0])

'00181'

In [19]:
#Training the Model
simple_logger = LambdaCallback(
    on_epoch_end=lambda e, l: print('{:.2f}'.format(l['val_accuracy']), end=' _ ')
)
early_stopping = EarlyStopping(monitor='val_loss', patience=10)

model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose=False,
         batch_size=1024, callbacks=[simple_logger, early_stopping])

0.61 _ 0.62 _ 0.63 _ 0.64 _ 0.65 _ 0.65 _ 0.67 _ 0.69 _ 0.71 _ 0.74 _ 0.75 _ 0.77 _ 0.78 _ 0.79 _ 0.79 _ 0.80 _ 0.81 _ 0.83 _ 0.85 _ 0.85 _ 0.89 _ 0.91 _ 0.92 _ 0.94 _ 0.95 _ 0.96 _ 0.96 _ 0.97 _ 0.97 _ 0.97 _ 0.98 _ 0.98 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 0.99 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 1.00 _ 

In [20]:
#Create a test set and look at some predictions
x_test, y_test = create_dataset(num_examples=20)
preds = model.predict(x_test)
full_seq_acc = 0

for i, pred in enumerate(preds):
    pred_str = devectorize_example(pred)
    y_test_str = devectorize_example(y_test[i])
    x_test_str = devectorize_example(x_test[i])
    col = 'green' if pred_str == y_test_str else 'red'
    full_seq_acc += 1/len(preds) * int(pred_str == y_test_str)
    outstring = 'Input: {}, Out: {}, Pred: {}'.format(x_test_str, y_test_str, pred_str)
    print(colored(outstring, col))
print('\nFull sequence accuracy: {:.3f} %'.format(100 * full_seq_acc))

Input: 08+52, Out: 00060, Pred: 00060
Input: 12+94, Out: 00106, Pred: 00106
Input: 03+55, Out: 00058, Pred: 00058
Input: 96+27, Out: 00123, Pred: 00123
Input: 28+47, Out: 00075, Pred: 00075
Input: 19+95, Out: 00114, Pred: 00114
Input: 60+96, Out: 00156, Pred: 00156
Input: 54+83, Out: 00137, Pred: 00137
Input: 79+46, Out: 00125, Pred: 00125
Input: 70+35, Out: 00105, Pred: 00105
Input: 79+72, Out: 00151, Pred: 00151
Input: 21+90, Out: 00111, Pred: 00111
Input: 18+44, Out: 00062, Pred: 00062
Input: 67+24, Out: 00091, Pred: 00091
Input: 65+64, Out: 00129, Pred: 00129
Input: 13+91, Out: 00104, Pred: 00104
Input: 16+54, Out: 00070, Pred: 00070
Input: 025+2, Out: 00027, Pred: 00027
Input: 010+4, Out: 00014, Pred: 00014
Input: 33+26, Out: 00059, Pred: 00059

Full sequence accuracy: 100.000 %
